# Red Inicial


## Imports


In [57]:
from collections import deque
from enum import Enum
from functools import cache
from random import random
from time import perf_counter, time
from typing import Generator, Optional

import gymnasium as gym
import numpy as np
from gymnasium.spaces import Box, Dict, Discrete
from gymnasium.utils import seeding
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env

## Clases y Funciones


In [58]:
class Packet_Generator():
    def __init__(self, min_ip=0, max_ip=2000,
                 min_port=0, max_port=4000,
                 min_protocol=0, max_protocol=100,
                 min_size=3, max_size=20,
                 min_rate=0, max_rate=4):

        self.packet = Dict({
            "IP":  Box(low=min_ip, high=max_ip, shape=(), dtype=int),
            "PORT":  Box(low=min_port, high=max_port, shape=(), dtype=int),
            "PROTOCOL":  Box(low=min_protocol, high=max_protocol, shape=(), dtype=int),
            "SIZE":  Box(low=min_size, high=max_size, shape=(), dtype=int)
        })
        self.min_rate: int = min_rate
        self.max_rate: int = max_rate

    def generate_packet(self):
        return self.packet.sample()

    def generate_packets(self):
        num_packets: int = np.random.randint(self.min_rate, self.max_rate)
        return [self.generate_packet() for _ in range(num_packets)]


class DOS_Packet_Generator(Packet_Generator):
    def __init__(self,
                 min_ip=0, max_ip=2000,
                 min_port=0, max_port=4000,
                 min_protocol=0, max_protocol=100,
                 min_size=10, max_size=20,
                 min_rate=20, max_rate=100):

        ip: int = np.random.randint(min_ip, max_ip)
        super().__init__(ip, ip,
                         min_port, max_port,
                         min_protocol, max_protocol,
                         min_size, max_size,
                         min_rate, max_rate)


class DDOS_Packet_Generator(Packet_Generator):
    def __init__(self,
                 min_ip=0, max_ip=2000,
                 min_port=0, max_port=4000,
                 min_protocol=0, max_protocol=100,
                 min_size=5, max_size=15,
                 min_rate=2, max_rate=10):

        super().__init__(min_ip, max_ip,
                         min_port, max_port,
                         min_protocol, max_protocol,
                         min_size, max_size,
                         min_rate, max_rate)


class PacketAttack(Enum):
    @staticmethod
    def new_set(description, weight, class_ref):
        return {
            "Description": description,
            "weight": weight,
            "class": class_ref
        }

    @staticmethod
    def not_implemented():
        raise NotImplementedError(f"Class not implemented")

    @classmethod
    @cache
    def weights(cls):
        attack_weights = []
        for attack in PacketAttack:
            attack_weights.append(attack.value["weight"])
        return np.array(attack_weights)

    # ----ENUM VALUES----
    DOS = new_set("Denial of Service", 1.0, DOS_Packet_Generator)
    DDOS = new_set("Distributed Denial of Service", 2.0, DDOS_Packet_Generator)


dos_gen = DOS_Packet_Generator()

print(dos_gen.generate_packets())

[{'IP': 722, 'PORT': 2509, 'PROTOCOL': 71, 'SIZE': 11}, {'IP': 722, 'PORT': 411, 'PROTOCOL': 84, 'SIZE': 12}, {'IP': 722, 'PORT': 3362, 'PROTOCOL': 7, 'SIZE': 13}, {'IP': 722, 'PORT': 1218, 'PROTOCOL': 6, 'SIZE': 16}, {'IP': 722, 'PORT': 2176, 'PROTOCOL': 91, 'SIZE': 18}, {'IP': 722, 'PORT': 3858, 'PROTOCOL': 36, 'SIZE': 11}, {'IP': 722, 'PORT': 591, 'PROTOCOL': 84, 'SIZE': 15}, {'IP': 722, 'PORT': 3648, 'PROTOCOL': 23, 'SIZE': 18}, {'IP': 722, 'PORT': 2315, 'PROTOCOL': 59, 'SIZE': 12}, {'IP': 722, 'PORT': 188, 'PROTOCOL': 2, 'SIZE': 17}, {'IP': 722, 'PORT': 1165, 'PROTOCOL': 42, 'SIZE': 20}, {'IP': 722, 'PORT': 2138, 'PROTOCOL': 72, 'SIZE': 13}, {'IP': 722, 'PORT': 2464, 'PROTOCOL': 5, 'SIZE': 17}, {'IP': 722, 'PORT': 4, 'PROTOCOL': 73, 'SIZE': 14}, {'IP': 722, 'PORT': 3977, 'PROTOCOL': 70, 'SIZE': 19}, {'IP': 722, 'PORT': 1311, 'PROTOCOL': 42, 'SIZE': 10}, {'IP': 722, 'PORT': 2607, 'PROTOCOL': 33, 'SIZE': 13}, {'IP': 722, 'PORT': 1678, 'PROTOCOL': 87, 'SIZE': 10}, {'IP': 722, 'PORT':

In [59]:
from time import perf_counter
tiempo_global: float = 0.0
iteraciones = 0


def medir_tiempo(activado=False):
    def fun(funcion):
        def wrapper(*args, **kwargs):
            if not activado:
                return funcion(*args, **kwargs)
            fun_tiempo = perf_counter
            inicio: float = fun_tiempo()
            resultado = funcion(*args, **kwargs)
            tiempo_total: float = fun_tiempo() - inicio
            global tiempo_global, iteraciones
            medida = 1e6
            t: float = tiempo_total*medida
            tiempo_global += t
            iteraciones += 1
            print(f"Tiempo de ejecución de {
                funcion.__name__}: {t:.2f} micro-segundos")
            return resultado
        return wrapper
    return fun


In [68]:
class Acciones(Enum):
    PERMITIR = 20
    DENEGAR = -20

    @classmethod
    def int_to_action(cls, action: int) -> "Acciones":
        return cls._get_actions_list()[action]

    @classmethod
    @cache
    def _get_actions_list(cls) -> list["Acciones"]:
        return list(Acciones)


class RouterEnv(gym.Env):
    total_time: float = 10.0  # En uds

    def __init__(self, max_len=20, seed: Optional[int] = None):

        super(RouterEnv, self).__init__()
        if max_len < 1:
            raise ValueError("max_len must be greater than 0")

        self.max_len: int = max_len
        self.rate = 20  # bytes por segundo de procesamiento
        self.attack_probability = 0.3

        self._set_initial_values(seed)

        self.observation_space = Box(low=0, high=1000, shape=(
            # Paquetes en la cola
            len(self.calculate_queue_stats()),), dtype=np.uint16)

        self.action_space = Discrete(len(Acciones))

    def _set_initial_values(self, seed):
        self.queue = deque(maxlen=self.max_len)
        self.step_durations: list[float] = []
        self._np_random, self._np_random_seed = seeding.np_random(seed)
        self.current_action: Acciones = Acciones.PERMITIR
        self.uds_tiempo_pasado: float = 0.0
        self.mb_restantes: float = -1.0

    def reset(self, seed: Optional[int] = None, options: Optional[dict] = None):
        self._set_initial_values(seed)

        observation = self._get_obs()
        info = self._get_info()
        return observation, info

    def _get_obs(self):
        stats = self.calculate_queue_stats()
        return stats

    def _get_info(self):
        npack, tam_total, *_ = self.calculate_queue_stats()
        return {"Stats": {
            "Queue": np.array(self.queue),
            "AttackProb": self.attack_probability,
            "NumPaquetes": npack,
            "TamañoTotal": tam_total,
            "Action": self.current_action
        }}

    def calculate_queue_stats(self):
        tam_total = 0
        for paquete in self.queue:
            tam_total += paquete["SIZE"]
        num_packets: int = len(self.queue)
        return np.array([num_packets, tam_total], dtype=np.uint16)

    def packet_input(self, input: list[dict[str, any]] = None) -> int:
        if input is not None:
            paquetes = input
        else:
            prob: float = self.np_random.random()
            if prob > self.attack_probability:
                # Generación de paquetes normales
                paquetes = Packet_Generator().generate_packets()
            else:
                # Generación de paquetes maliciosos
                paquetes = DOS_Packet_Generator().generate_packets()

        if len(self.queue) + len(paquetes) > self.max_len:
            espacio_libre = self.max_len - len(self.queue)
            self.queue.extend(paquetes[:espacio_libre])
            return len(paquetes) - (espacio_libre)
        self.queue.extend(paquetes)
        return 0  # No se han descartado paquetes

    @medir_tiempo(0)
    def step(self, action: int):
        # Prob
        # Hacer procesado por tamaño de paquete
        # Cada paso debería ser 10 microsegundos

        # Suponiendo que el paso es de 10 microsegundos:
        print("Nuevo step")
        print(self.current_action)
        action = Acciones.int_to_action(action)
        self.current_action = action
        if self.current_action == Acciones.PERMITIR:
            descartados: int = self.packet_input()
        else:
            descartados: int = 0

        self.procesar_por_tamaño()

        reward: float = self.get_reward(descartados, action)
        observation = self._get_obs()
        # True si se desvía del comportamiento normal para abortar, necesitaría un reset
        truncated = False
        info = self._get_info()

        self.uds_tiempo_pasado += 1
        finished: bool = self._is_finished_execution()
        if len(self.queue) == 0:
            print("Ha terminado")
        # self.step_durations.append(time() - start_time)
        return observation, reward, finished, truncated, info

    def get_reward(self, descartados, action) -> float:
        debu = []

        def calcularOcupacion() -> float:
            return len(self.queue) / self.max_len

        ocupacion_media = 0.8
        reward = 10

        actual: float = calcularOcupacion()

        if actual > ocupacion_media:
            penalizacion_ocupacion: float = (((actual - ocupacion_media)*self.max_len )** 2)
            max = (1 - ocupacion_media) ** 2
            #penalizacion_ocupacion es menor que 0
            penalizacion_ocupacion
            if reward > 0:
                reward -= penalizacion_ocupacion
            else:
                reward += penalizacion_ocupacion
            # debu.append(-penalizacion_ocupacion)

        debu.append(reward)

        penalizacion_descartados = 2
        reward -= descartados*penalizacion_descartados

        debu.append(-descartados*penalizacion_descartados)

        reward += action.value
        debu.append(action.value)
        print("Recompensas", debu, sum(debu))
        reward=self.np_random.random()
        return reward

    @medir_tiempo(0)
    def procesar_por_tamaño(self):

        if len(self.queue) == 0:
            return

        # print(self.queue)
        tam_procesado = 0.0
        # Calcula los mb que faltan por procesar
        """
        """
        if self.mb_restantes == -1 and len(self.queue) > 0:
            paquete = self.queue[0]
            self.mb_restantes = paquete["SIZE"]

        # TODO mirar lo de mb restantes
        while tam_procesado < self.rate and len(self.queue) > 0:
            # print(f"{self.mb_restantes} MB restantes")
            if self.mb_restantes == 0:
                p2 = self.queue.popleft()  # Quita el paquete que se ha procesado
                if len(self.queue) == 0:
                    self.mb_restantes == -1
                    break
                # Nuevo paquete
                paquete = self.queue[0]
                assert p2 != paquete
                # Calcula los mb que faltan por procesar
                self.mb_restantes = paquete["SIZE"]
            else:
                # Procesar
                procesado_local: float = min(self.mb_restantes,  # Procesar lo que queda del paquete
                                             self.rate-tam_procesado)  # Procesar lo que queda del paso
                self.mb_restantes -= procesado_local
                tam_procesado += procesado_local

        # print(f"Procesado {tam_procesado} bytes")

    def _is_finished_execution(self) -> bool:
        # Terminar solo después de 10 pasos
        return self.uds_tiempo_pasado >= self.total_time
        return len(self.step_durations) >= 10

    def close(self):
        # Cerrar el entorno, liberar recursos, cerrar conexiones, etc
        return super().close()

    def render(self, mode='human'):
        # Renderizar el entorno
        return super().render(mode=mode)

In [69]:
env = RouterEnv(seed=1)
check_env(env)

model:PPO = PPO("MlpPolicy", env, verbose=True)

model.learn(total_timesteps=10)
model.save("Example")

Nuevo step
Acciones.PERMITIR
Recompensas [10, 0, -20] -10
Ha terminado
Nuevo step
Acciones.PERMITIR
Recompensas [10, 0, -20] -10
Ha terminado
Nuevo step
Acciones.DENEGAR
Recompensas [10, 0, 20] 30
Ha terminado
Nuevo step
Acciones.PERMITIR
Recompensas [10, 0, -20] -10
Ha terminado
Nuevo step
Acciones.DENEGAR
Recompensas [10, 0, -20] -10
Ha terminado
Nuevo step
Acciones.DENEGAR
Recompensas [-5.999999999999993, -108, 20] -94.0
Nuevo step
Acciones.PERMITIR
Recompensas [6.000000000000002, -2, 20] 24.0
Nuevo step
Acciones.PERMITIR
Recompensas [6.000000000000002, 0, 20] 26.0
Nuevo step
Acciones.PERMITIR
Recompensas [6.000000000000002, -2, 20] 24.0
Nuevo step
Acciones.PERMITIR
Recompensas [9.000000000000004, 0, 20] 29.000000000000004
Nuevo step
Acciones.PERMITIR
Recompensas [9.000000000000004, 0, 20] 29.000000000000004
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Nuevo step
Acciones.PERMITIR
Recompensas [10, 0, -20] -10
Ha terminado
Nuevo step
A

In [70]:
print(f"Tiempo medio: {tiempo_global/iteraciones if abs(iteraciones)>1e-5 else 0:.2f} micro-segundos de {iteraciones} iteraciones")

Tiempo medio: 0.00 micro-segundos de 0 iteraciones


## Comprobación funcionamiento

In [80]:
env = RouterEnv(seed=None)
model = PPO.load("Example")

num_steps = 3
obs, _ = env.reset()

acciones = []
for episode in range(num_steps):

    done = False
    step_counter = 0
    while not done:
        action, _states = model.predict(obs, deterministic=True)
        obs, rewards, done, terminated, info = env.step(action)
        acciones.append(action.item())
        print(info.get("Stats", info))
        print(obs, rewards)

        done: bool = done or terminated

    step_counter += 1
print(acciones)
print(list(map(lambda x: Acciones.int_to_action(x), acciones)))

Nuevo step
Acciones.PERMITIR
Recompensas [10, 0, 20] 30
Ha terminado
{'Queue': array([], dtype=float64), 'AttackProb': 0.3, 'NumPaquetes': 0, 'TamañoTotal': 0, 'Action': <Acciones.PERMITIR: 20>}
[0 0] 0.03478024316029582
Nuevo step
Acciones.PERMITIR
Recompensas [1.0000000000000107, -154, 20] -133.0
{'Queue': array([{'IP': 1231, 'PORT': 1405, 'PROTOCOL': 91, 'SIZE': 13},
       {'IP': 1231, 'PORT': 3514, 'PROTOCOL': 39, 'SIZE': 20},
       {'IP': 1231, 'PORT': 2633, 'PROTOCOL': 62, 'SIZE': 17},
       {'IP': 1231, 'PORT': 3141, 'PROTOCOL': 80, 'SIZE': 18},
       {'IP': 1231, 'PORT': 3892, 'PROTOCOL': 85, 'SIZE': 16},
       {'IP': 1231, 'PORT': 3136, 'PROTOCOL': 11, 'SIZE': 20},
       {'IP': 1231, 'PORT': 3517, 'PROTOCOL': 83, 'SIZE': 14},
       {'IP': 1231, 'PORT': 3942, 'PROTOCOL': 38, 'SIZE': 16},
       {'IP': 1231, 'PORT': 3362, 'PROTOCOL': 1, 'SIZE': 16},
       {'IP': 1231, 'PORT': 3942, 'PROTOCOL': 54, 'SIZE': 18},
       {'IP': 1231, 'PORT': 3562, 'PROTOCOL': 85, 'SIZE': 20}